# Exercise: Babylonian Square Root

The Babylonians introduced a simple but clever iterative algorithm for computing the square root of a number:

> Repeat $t \leftarrow (t + x/2)/2$ until $t$ converges to $\sqrt{x}$.

Here a Julia implementation that performs the iterative procedure `N` times.

In [1]:
function sqrt_babylonian(x; N = 10)
    t = (1+x)/2
    for i = 2:N
        t = (t + x/t)/2
    end
    t
end

sqrt_babylonian (generic function with 1 method)

### Tasks

1. Confirm that the iterative algorithm indeed converges to the square root by comparing the output of `sqrt_babylonian` to `sqrt(big"2.0")` for `N` increasing from `1` to `10`.
    - How big is the difference to the "exact" result?
    - Does increasing `N` any further help?
    - How can we get a more accurate result without changing the implementation?
2. Do the same as in task 1 but vary the data type of the input number. Specifically, use `Float16`, `Float32`, `Float64`, and `BigFloat`.

We can obtain fancy features just by changing the input argument type. A powerful number type invented by Clifford in 1873 is the [*dual number*](https://en.wikipedia.org/wiki/Dual_number). One application of these numbers is what is known as forward-mode automatic differentiation (AD) these days. Here a simple (incomplete) Julia implementation:

In [2]:
# D for "dual number", invented by Clifford in 1873.
struct D <: Number
    x::Float64 # value
    ϵ::Float64 # derivative
end

Base.:+(a::D, b::D) = D(a.x + b.x, a.ϵ + b.ϵ) # sum rule
Base.:-(a::D, b::D) = D(a.x - b.x, a.ϵ - b.ϵ) # sum rule
Base.:*(a::D, b::D) = D(a.x * b.x, a.x * b.ϵ + a.ϵ * b.x) # product rule
Base.:/(a::D, b::D) = D(a.x / b.x, (b.x * a.ϵ - a.x * b.ϵ)/b.x^2) # quotient rule

Base.convert(::Type{D}, x::Real) = D(x, zero(x))
Base.promote_rule(::Type{D}, ::Type{<:Number}) = D

The derivative of a generic Julia function (that fundamentally relies on `+`, `-`, `*`, and `/`) can know be obtained with the following function

In [3]:
# utility function for our small forward-mode AD
derivative(f::Function, x::Number) = f(D(x, one(x))).ϵ

derivative (generic function with 1 method)

3. What is the derivative of $\sqrt{x}$?

4. Check that `derivative(sqrt_babylonian, some_number)` indeed automagically gives the correct value of the derivative.
    - Try other functions/algorithms as well! (e.g. maybe something recursive like `pow(x, n) = n <= 0 ? 1 : x*pow(x, n-1)`)


Now, imagine your boss coming to your office and telling you that he thinks that the Babylonian approximation for `N=4` is equivalent to the analytical form
$$ \text{sqrt_babylonian}(x; N=4) \approx \begin{equation}
\frac{\frac{1}{32768} + \frac{15}{4096} x + \frac{455}{8192} x^{2} + \frac{15}{4096} x^{7} + \frac{455}{8192} x^{6} + \frac{1001}{4096} x^{3} + \frac{6435}{16384} x^{4} + \frac{1001}{4096} x^{5} + \frac{1}{32768} x^{8}}{\left( \frac{1}{2} + \frac{1}{2} x \right) \left( \frac{1}{8} + \frac{1}{8} x^{2} + \frac{3}{4} x \right) \left( \frac{1}{128} + \frac{1}{128} x^{4} + \frac{7}{32} x + \frac{35}{64} x^{2} + \frac{7}{32} x^{3} \right)}
\end{equation} $$

Of course, he asks you to "quickly" confirm this.

5. Is your boss correct?
    - Hint: Use the `Symbolics` package, in particular `@variables x` and `simplify`.
    - Note: Make sure you have `N=4` since for larger `N` problems might start to occur.

### Possible solution

In [4]:
# 1.
for i in 1:10
    println(sqrt_babylonian(2.0; N=i) - sqrt(big"2.0"))
end

0.08578643762690495119831127579030192143032812462305192682332026200926752153789802
0.00245310429357146983524132576942986494610571251367692682332026200926752153789802
2.123901414615797774852241245868268707030873051926823320262009267521537898020485e-06
1.594821025504709383794876399734252552739426823320262009267521537898020485288439e-12
-1.253716717905021777128546450199081980731766797379907324784621019795147115605803e-16
-1.253716717905021777128546450199081980731766797379907324784621019795147115605803e-16
-1.253716717905021777128546450199081980731766797379907324784621019795147115605803e-16
-1.253716717905021777128546450199081980731766797379907324784621019795147115605803e-16
-1.253716717905021777128546450199081980731766797379907324784621019795147115605803e-16
-1.253716717905021777128546450199081980731766797379907324784621019795147115605803e-16


In [5]:
# 2.
for dtype in (Float16, Float32, Float64, BigFloat)
    println("dtype = $dtype")
    for i in 1:10
        println(sqrt_babylonian(dtype(2.0); N=i) - sqrt(big"2.0"))
    end
    println()
end

dtype = Float16
0.08578643762690495119831127579030192143032812462305192682332026200926752153789802
0.00180206262690495119831127579030192143032812462305192682332026200926752153789802
-0.0001510623730950488016887242096980785696718753769480731766797379907324784621019795
-0.0001510623730950488016887242096980785696718753769480731766797379907324784621019795
-0.0001510623730950488016887242096980785696718753769480731766797379907324784621019795
-0.0001510623730950488016887242096980785696718753769480731766797379907324784621019795
-0.0001510623730950488016887242096980785696718753769480731766797379907324784621019795
-0.0001510623730950488016887242096980785696718753769480731766797379907324784621019795
-0.0001510623730950488016887242096980785696718753769480731766797379907324784621019795
-0.0001510623730950488016887242096980785696718753769480731766797379907324784621019795

dtype = Float32
0.08578643762690495119831127579030192143032812462305192682332026200926752153789802
0.0024531837664313183858112757

In [6]:
# 3.
derivative(sqrt_babylonian, 2.0)

0.35355339059327373

Derivative of $\sqrt{x}$ is $\dfrac{1}{2\sqrt{x}}$

In [7]:
1/(2*sqrt(2))

0.35355339059327373

In [8]:
derivative(x->2*x^2, 1.23) == 4*1.23

true

In [9]:
pow(x, n) = n <= 0 ? 1 : x*pow(x, n-1)

pow (generic function with 1 method)

In [10]:
derivative(x -> pow(x,3), 2)

12.0

In [11]:
# X.
using Symbolics
@variables x

for i in 1:4
    display(sqrt_babylonian(x; N=i))
end

(1//2) + (1//2)*x

(1//4) + (1//4)*x + ((1//2)*x) / ((1//2) + (1//2)*x)

(1//8) + (1//8)*x + ((1//4)*x) / ((1//2) + (1//2)*x) + ((1//2)*x) / ((1//4) + (1//4)*x + ((1//2)*x) / ((1//2) + (1//2)*x))

(1//16) + (1//16)*x + ((1//8)*x) / ((1//2) + (1//2)*x) + ((1//4)*x) / ((1//4) + (1//4)*x + ((1//2)*x) / ((1//2) + (1//2)*x)) + ((1//2)*x) / ((1//8) + (1//8)*x + ((1//4)*x) / ((1//2) + (1//2)*x) + ((1//2)*x) / ((1//4) + (1//4)*x + ((1//2)*x) / ((1//2) + (1//2)*x)))

In [26]:
simplify(sqrt_babylonian(x; N=4))

((1//32768) + (15//4096)*x + (455//8192)*(x^2) + (15//4096)*(x^7) + (455//8192)*(x^6) + (1001//4096)*(x^3) + (6435//16384)*(x^4) + (1001//4096)*(x^5) + (1//32768)*(x^8)) / (((1//2) + (1//2)*x)*((1//8) + (1//8)*(x^2) + (3//4)*x)*((1//128) + (1//128)*(x^4) + (7//32)*x + (35//64)*(x^2) + (7//32)*(x^3)))